In [ ]:
# ws03
import numpy as np

from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']

news = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True)
news = fetch_20newsgroups(subset='train', categories=categories)


In [ ]:
news.keys()

In [ ]:
print(news.DESCR)

In [ ]:
len(news.target_names)

In [ ]:
len(news.data)

In [ ]:
len(news.target)

In [ ]:
news.target_names

In [ ]:
news.target[:10]

In [ ]:
for k in news.target[:10]:
    print(news.target_names[k])

# Histogram

In [ ]:
np.unique(news.target)

In [ ]:
labels, counts = np.unique(news.target, return_counts=True)

labels_str = np.array(news.target_names)[labels]

list(zip(labels_str, counts))

In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(labels_str, counts)))
df

In [ ]:
%matplotlib inline
import seaborn as sns
sns.countplot(news.target)

In [ ]:
# ws04

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

text = news.data
cv = CountVectorizer()
cv = CountVectorizer(stop_words='english')
count_matrix = cv.fit_transform(text)

word_matrix = count_matrix.toarray()  #
# word_matrix = count_matrix.todense()  # 
df = pd.DataFrame(word_matrix, 
                  columns=cv.get_feature_names())

df.head()

In [ ]:
df.shape

In [ ]:
df[df.columns[10000:10010]].head()

In [ ]:
dfsum = df.sum(axis=0, skipna=True)
bow = dfsum.sort_values(ascending=False)[:30]
bow 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.subplots(figsize=(3, 4))
sns.barplot(bow.values[:20], bow.index[:20])

plt.ylabel('Words')
plt.xlabel('Frequencies')
plt.title('Frequency Distribution')
plt.show()

## ws05

In [ ]:
print(news.data[0])

In [ ]:
news.target[:10] 
news.target[0] 

In [ ]:
news.target_names[(news.target[0])]

In [ ]:
for k in news.target[:10]:
    print(news.target_names[k])

## ws06

In [ ]:
# Train Test Split
X = news.data
y = news.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

In [ ]:
len(X_train),len(X_test)

In [ ]:
import time
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

start = time.time()

tvec = TfidfVectorizer()
X_train_tfidf = tvec.fit_transform(X_train)
X_test_tfidf = tvec.transform(X_test)

model = MultinomialNB(alpha=0.005)
model.fit(X_train_tfidf, y_train)

end = time.time()

print("Accuracy: {:.3f} , Time: {:.3f} ".
      format(model.score(X_test_tfidf, y_test), (end - start)))

## ws07

In [ ]:
# Predict
new_docs = ['OpenGL on the GPU is fast.']
new_docs = ['ask about health care topic.']
new_docs = ['God is love. loves you every body.']

new_docs_cv = tvec.transform(new_docs)
y_pred = model.predict(new_docs_cv)

news.target_names[y_pred[0]]

In [ ]:
# Predict
new_docs = ['God is love. loves you every body.',
           'OpenGL on the GPU is fast.',
           'ask about health care topic.']

new_docs_cv = tvec.transform(new_docs)

y_pred = model.predict(new_docs_cv)
y_pred

for k in y_pred:
    print(news.target_names[k]) 

## ws08 Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

import time
start = time.time()

pipe1 = Pipeline([('vectorizer', TfidfVectorizer()),   # 975
                  ('classifier', MultinomialNB(alpha=0.005))])

pipe1.fit(X_train, y_train)
end = time.time()

print("Accuracy: {:.3f} , Time: {:.3f} ".
      format(pipe1.score(X_test, y_test), (end - start)))

In [ ]:
# 
new_docs = ['God is love. loves you every body.',
           'OpenGL on the GPU is fast.',
           'ask about health care topic.']

y_pred = pipe1.predict(new_docs)
y_pred

for k in y_pred:
    print(news.target_names[k]) 
    

In [ ]:
new_doc = 'I would like to know a quick way\
to convert .ps files into LaserJet III format.  \
the same, converting to HPGL (HP plotter) files. \
Please email any response.'

y_pred = pipe1.predict([new_doc])

news.target_names[y_pred[0]]

## ws09

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

y_pred = pipe1.predict(X_test)
labels = news.target_names

print(metrics.classification_report(y_test, y_pred, target_names=labels))

cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.metrics import confusion_matrix # Confusion Matrix
import matplotlib.pyplot as plt
y_pred = pipe1.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
labels=news.target_names
fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(cm, annot=True, cmap='Set3', fmt ='d', 
    xticklabels=labels, yticklabels=labels)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()